In [1]:
!pip install -q scikit-learn gradio pandas

In [6]:
import pandas as pd
import pickle
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

print("📥 Loading churn dataset...")

# Load dataset
data = fetch_openml(data_id=40701, as_frame=True)
df = data.frame

print(f"✅ Columns in dataset: {df.columns.tolist()}")

# Drop rows with missing
df.dropna(inplace=True)

# Encode categorical columns
target_col = 'class'

for col in df.select_dtypes('object').columns:
    if col != target_col:
        df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop(target_col, axis=1)
y = LabelEncoder().fit_transform(df[target_col])  # encode target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save model + feature names
with open("churn_model.pkl", "wb") as f:
    pickle.dump((clf, list(X.columns)), f)

print("✅ churn_model.pkl saved.")


📥 Loading churn dataset...
✅ Columns in dataset: ['state', 'account_length', 'area_code', 'phone_number', 'international_plan', 'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls', 'class']
✅ churn_model.pkl saved.


In [9]:
df.head()

,state,account_length,area_code,phone_number,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,class
0,16,128,415,2845,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,35,107,415,2301,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,31,137,415,1616,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,35,84,408,2510,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,36,75,415,155,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [10]:
# Print the first row completely
print("\n📄 First row values:")
print(df.iloc[0])


📄 First row values:
state                               16
account_length                     128
area_code                          415
phone_number                      2845
international_plan                   0
voice_mail_plan                      1
number_vmail_messages               25
total_day_minutes                265.1
total_day_calls                    110
total_day_charge                 45.07
total_eve_minutes                197.4
total_eve_calls                     99
total_eve_charge                 16.78
total_night_minutes              244.7
total_night_calls                   91
total_night_charge               11.01
total_intl_minutes                10.0
total_intl_calls                     3
total_intl_charge                  2.7
number_customer_service_calls        1
class                                0
Name: 0, dtype: object


In [7]:
import numpy as np

print("📝 Writing app.py and requirements.txt...")

# app.py content
app_py = f"""
import pickle
import gradio as gr
import numpy as np

with open("churn_model.pkl", "rb") as f:
    clf, feature_names = pickle.load(f)

def predict_churn(*features):
    X = np.array(features).reshape(1, -1)
    pred = clf.predict(X)[0]
    return "Yes" if pred == 1 else "No"

def chatbot(user_message, history=[]):
    history = history or []
    user_message = user_message.lower()
    if "churn" in user_message:
        reply = "You can use the inputs on the left to check if a customer is likely to churn!"
    elif "hello" in user_message or "hi" in user_message:
        reply = "Hello! How can I help you today?"
    elif "bye" in user_message:
        reply = "Goodbye! Have a nice day!"
    else:
        reply = "I'm here to help you with Churn predictions. Try asking about that!"
    history.append((user_message, reply))
    return history, history

inputs = [gr.Number(label=feat) for feat in feature_names]
output = gr.Textbox(label="Will the customer churn?")

prediction_interface = gr.Interface(
    fn=predict_churn,
    inputs=inputs,
    outputs=output,
    title="Customer Churn Predictor",
    description="Enter customer details to predict churn."
)

chatbot_interface = gr.ChatInterface(
    fn=chatbot,
    title="Churn Assistant Bot",
    description="Ask me anything about the churn predictor!"
)

with gr.Blocks() as demo:
    gr.Markdown("# 🔮 Customer Churn Prediction + Chatbot")
    with gr.Row():
        with gr.Column():
            prediction_interface.render()
        with gr.Column():
            chatbot_interface.render()

demo.launch()
"""

# Write app.py
with open("app.py", "w") as f:
    f.write(app_py.strip())

# Write requirements.txt
req_txt = """
scikit-learn
pandas
numpy
gradio
"""

with open("requirements.txt", "w") as f:
    f.write(req_txt.strip())

print("✅ app.py and requirements.txt saved.")


📝 Writing app.py and requirements.txt...
✅ app.py and requirements.txt saved.


In [8]:
import pickle
import gradio as gr
import numpy as np

with open("churn_model.pkl", "rb") as f:
    clf, feature_names = pickle.load(f)

def predict_churn(*features):
    X = np.array(features).reshape(1, -1)
    pred = clf.predict(X)[0]
    return "Yes" if pred == 1 else "No"

def chatbot(user_message, history=[]):
    history = history or []
    user_message = user_message.lower()
    if "churn" in user_message:
        reply = "You can use the inputs on the left to check if a customer is likely to churn!"
    elif "hello" in user_message or "hi" in user_message:
        reply = "Hello! How can I help you today?"
    elif "bye" in user_message:
        reply = "Goodbye! Have a nice day!"
    else:
        reply = "I'm here to help you with Churn predictions. Try asking about that!"
    history.append((user_message, reply))
    return history, history

inputs = [gr.Number(label=feat) for feat in feature_names]
output = gr.Textbox(label="Will the customer churn?")

prediction_interface = gr.Interface(
    fn=predict_churn,
    inputs=inputs,
    outputs=output,
    title="Customer Churn Predictor",
    description="Enter customer details to predict churn."
)

chatbot_interface = gr.ChatInterface(
    fn=chatbot,
    title="Churn Assistant Bot",
    description="Ask me anything about the churn predictor!"
)

with gr.Blocks() as demo:
    gr.Markdown("# 🔮 Customer Churn Prediction + Chatbot")
    with gr.Row():
        with gr.Column():
            prediction_interface.render()
        with gr.Column():
            chatbot_interface.render()

demo.launch(share=True)  # share=True to get a public link


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ed168fc4ada059ccd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In this dataset:

📈 international_plan → HIGHLY correlated
(People with an international plan churn more often — makes sense, it’s expensive & high-maintenance.)

📈 number_customer_service_calls → strongly related
(Customers who call support many times are more likely to churn.)

📈 total_day_minutes or total_day_charge → moderate correlation
(More usage can correlate with churn depending on satisfaction.)

🎯 Customers with international plans and high support calls are at highest risk of churn → these customers should be proactively engaged.

🎯 Reducing issues that lead to many customer service calls can help reduce churn.

🎯 Customers with voice mail plan are more loyal → promoting this could help retain customers.

